In [1]:
import nltk
import gensim
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora,models
import os
import re

In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('Data/train.csv')

In [4]:
match_patt=r'@[0-9a-z]+|https://\w+[./\w]+|http://\w+[./\w]+|bit.ly[/\.\w]+|[^a-z]|[\d]+'
# hashtag '#[0-9a-z_]+'
# mention '@[0-9a-z]+'
# website 'https://\w+[./\w]+|http://\w+[./\w]+'
# short website 'bit.ly[/\.\w]+'
# punctuations '[^a-z]'
# numbers '[\d]+'
def find_pat(txt):
    return re.sub(match_patt,' ',txt,flags=re.I)

In [5]:
all_text=np.array(data['tweet'])

In [6]:
tokenizer=RegexpTokenizer(r'\w+')
en_stop_words=set(stopwords.words('english'))
p_stem=PorterStemmer()

In [7]:
def create_corpus(txt_data):
    text=[]
    for i in txt_data:
        tokens=tokenizer.tokenize(find_pat(str(i)).lower())
        stop_tok=[i for i in tokens if i not in en_stop_words]
        stem_tok=[p_stem.stem(i) for i in stop_tok]
        text.append(stem_tok)
    dictionary=corpora.Dictionary(text)
    corpus=[dictionary.doc2bow(t) for t in text]
    return text,dictionary,corpus

In [8]:
texts, dictionaries, txt_corpus=create_corpus(all_text)

In [9]:
lda_model=models.LdaMulticore(txt_corpus,num_topics=4,id2word=dictionaries,passes=279)

for l in lda_model.print_topics():
    print(l)
    print('')

import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model, txt_corpus, dictionaries)

vis

(0, '0.085*"sxsw" + 0.052*"googl" + 0.036*"quot" + 0.027*"link" + 0.025*"rt" + 0.009*"appl" + 0.009*"map" + 0.008*"amp" + 0.008*"mobil" + 0.006*"mayer"')

(1, '0.095*"sxsw" + 0.067*"ipad" + 0.067*"appl" + 0.051*"store" + 0.044*"link" + 0.032*"rt" + 0.026*"austin" + 0.022*"pop" + 0.018*"open" + 0.016*"line"')

(2, '0.076*"sxsw" + 0.055*"googl" + 0.051*"link" + 0.038*"circl" + 0.036*"social" + 0.035*"new" + 0.033*"launch" + 0.033*"rt" + 0.027*"today" + 0.026*"network"')

(3, '0.087*"sxsw" + 0.046*"iphon" + 0.037*"link" + 0.036*"app" + 0.024*"ipad" + 0.023*"rt" + 0.014*"android" + 0.013*"amp" + 0.012*"free" + 0.011*"new"')



PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.041509  0.147358       1        1  28.709368
0     -0.068739  0.092374       2        1  27.643433
1      0.193868 -0.110166       3        1  26.819691
2     -0.166639 -0.129566       4        1  16.827509, topic_info=      Term         Freq        Total Category  logprob  loglift
15   googl  1957.000000  1957.000000  Default  30.0000  30.0000
0     appl  1696.000000  1696.000000  Default  29.0000  29.0000
52   store  1138.000000  1138.000000  Default  28.0000  28.0000
43   iphon  1211.000000  1211.000000  Default  27.0000  27.0000
34     app   862.000000   862.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
66   video    50.275440   115.893967   Topic4  -5.6244   0.9470
441  sxswi    59.010887   274.832245   Topic4  -5.4642   0.2437
361   take    45.493484   125.304955   Topic4  -5.7244   0.7690
98     get    51.506432   375.106903   Topic4  -5.6002  -0.2033
103      w    42.802059   203.925415   Topic4  -5.7853   0.2210

[258 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
11        1  0.502201     amp
11        2  0.313673     amp
11        3  0.169026     amp
11        4  0.016252     amp
1100      2  0.984952  analyt
...     ...       ...     ...
672       1  0.760480   world
672       2  0.225768   world
252       1  0.392964    year
252       2  0.556700    year
252       3  0.052395    year

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3])